In [9]:
%pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [10]:
tickers = 'AAPL,TSLA,MSFT,GM,BCS,QCOM,PTR,VALE,GS,CI,ENB,CB,LFC,MUFG,NVDA,ASML,ADBE,SONY,AMD,TXN,F,'
indices = 'SPY,'

md = yf.download(
    tickers = indices + tickers,
    group_by='ticker',
    start='2020-05-19', end='2021-05-19',
    interval = '1d',
    auto_adjust = True,
    threads = True
    )

print(f'Downloaded {len(md)} days of market data for indices : {indices} and tickers : {tickers}')

md['SPY'].tail()
print(f"Index : {md.index[:5]}...")
print(f"Columns : {md.columns[:5]}...")



[*********************100%***********************]  22 of 22 completed
Downloaded 253 days of market data for indices : SPY, and tickers : AAPL,TSLA,MSFT,GM,BCS,QCOM,PTR,VALE,GS,CI,ENB,CB,LFC,MUFG,NVDA,ASML,ADBE,SONY,AMD,TXN,F,
Index : DatetimeIndex(['2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21',
               '2020-05-22'],
              dtype='datetime64[ns]', name='Date', freq=None)...
Columns : MultiIndex([('NVDA',   'Open'),
            ('NVDA',   'High'),
            ('NVDA',    'Low'),
            ('NVDA',  'Close'),
            ('NVDA', 'Volume')],
           )...


In [11]:
def getPriceAndReturn(md):
    priceField = 'Close'
    prices = pd.DataFrame()
    returns = pd.DataFrame()
    for ticker, kind in md.columns:
        if kind == priceField:
            prices[ticker] = md[ticker][[priceField]].values.reshape(1, -1).ravel()
            returns[ticker] = prices[ticker].pct_change()
    return prices, returns

prices, returns = getPriceAndReturn(md)

print(prices['SPY'].head())
print(returns['SPY'].head())



0    290.343842
1    287.361664
2    292.243408
3    290.225769
4    290.776947
Name: SPY, dtype: float64
0         NaN
1   -0.010271
2    0.016988
3   -0.006904
4    0.001899
Name: SPY, dtype: float64


In [12]:
def getStats(prices, returns):
    mean = []
    std = []
    sharpe = []
    totReturn = []

    for ticker in returns.columns:
        pr = prices[ticker]
        ret = returns[ticker]

        totReturn.append((pr.iloc[-1] - pr.iloc[0]) / pr.iloc[0])
        mean.append(ret.mean())
        std.append(ret.std())

        sharpe.append(mean[-1] / std[-1] * (252**0.5)) #TODO consider adding risk free rate

    return pd.DataFrame.from_dict(
        {'DailyMean' : mean, 'DailyStd' : std, 'AnualSR' : sharpe, 'TotalReturn' : totReturn},
        orient='index', columns=returns.columns)

stats = getStats(prices, returns)

print(f"Daily return SPY mean : {stats['SPY']['DailyMean']}")
print(f"Daily return SPY std  : {stats['SPY']['DailyStd']}")

print(f"Basic return stats (mean, std, sharpe ratio, total return) :\n{stats}")

Daily return SPY mean : 0.0014474968820360192
Daily return SPY std  : 0.010791323551446576
Basic return stats (mean, std, sharpe ratio, total return) :
                 NVDA       AMD        CI      QCOM       SPY      VALE  \
DailyMean    0.002248  0.001694  0.001461  0.002320  0.001447  0.004174   
DailyStd     0.027271  0.030751  0.021279  0.026079  0.010791  0.024523   
AnualSR      1.308776  0.874549  1.089657  1.412247  2.129331  2.701746   
TotalReturn  0.603992  0.363620  0.365772  0.648640  0.418801  1.650893   

                  BCS        GS      SONY       ENB  ...      ASML         F  \
DailyMean    0.003214  0.003071  0.001725  0.001179  ...  0.003211  0.003638   
DailyStd     0.030634  0.021120  0.018099  0.015030  ...  0.023965  0.026592   
AnualSR      1.665637  2.308167  1.512970  1.245412  ...  2.126632  2.171447   
TotalReturn  1.000275  1.048213  0.482753  0.308307  ...  1.087622  1.286252   

                   GM      ADBE      MSFT       PTR        CB      AAPL

In [13]:
def getCovAndCorr(returns, stats):
    cov = []
    corr = []

    for i in range(len(returns.columns)):
        covColumn = []
        corrColumn = []
        
        iname = returns.columns[i]
        for j in range(i):
            covColumn.append(cov[j][i])
            corrColumn.append(corr[j][i])
        iStd = stats[iname]["DailyStd"]
        covColumn.append(iStd * iStd)
        corrColumn.append(1)
        for j in range(i+1, len(returns.columns)):
            jname = returns.columns[j]
            jStd = stats[jname]["DailyStd"]
            tickerCov = returns[iname].cov(returns[jname])
            covColumn.append(tickerCov)
            corrColumn.append(tickerCov / (iStd * jStd))
        cov.append(covColumn)
        corr.append(corrColumn)

    covDf = pd.DataFrame(cov, index = returns.columns, columns = returns.columns)
    corrDf = pd.DataFrame(corr, index = returns.columns, columns = returns.columns)
    return covDf, corrDf

cov, corr = getCovAndCorr(returns, stats)

print(f"Correlation matrix :\n{corr}\n")
print(f"Test Correlation TSLA/MSFT: {returns['TSLA'].cov(returns['MSFT'])/(stats['TSLA']['DailyStd'] * stats['MSFT']['DailyStd'])}")
'''
print(f"Covariance :\n{cov}\n")
print(f"Test Covariance TSLA/MSFT: {returns['TSLA'].cov(returns['MSFT'])}")
print(f"Covarience SPY column :\n{cov['SPY']}")
print(f"Covarience TSLA row :\n{cov.loc['TSLA']}")
print(f"corr.loc['SPY','TSLA'] : {corr.loc['SPY','TSLA']}")
'''
print(f"corr.loc['TSLA','SPY'] : {corr.loc['TSLA','SPY']}")


Correlation matrix :
          NVDA       AMD        CI      QCOM       SPY      VALE       BCS  \
NVDA  1.000000  0.696549  0.078520  0.574012  0.591228  0.278942 -0.039333   
AMD   0.696549  1.000000  0.064646  0.490682  0.461708  0.271666 -0.046718   
CI    0.078520  0.064646  1.000000  0.173763  0.461357  0.178510  0.323919   
QCOM  0.574012  0.490682  0.173763  1.000000  0.598438  0.380698  0.151658   
SPY   0.591228  0.461708  0.461357  0.598438  1.000000  0.510540  0.473770   
VALE  0.278942  0.271666  0.178510  0.380698  0.510540  1.000000  0.369361   
BCS  -0.039333 -0.046718  0.323919  0.151658  0.473770  0.369361  1.000000   
GS    0.085137  0.051499  0.399901  0.226102  0.575174  0.395287  0.666330   
SONY  0.358647  0.263708  0.032099  0.343778  0.382444  0.335296  0.123734   
ENB   0.128385  0.165534  0.430018  0.249886  0.544283  0.445642  0.444674   
TSLA  0.514542  0.391549  0.009327  0.386461  0.464253  0.245919  0.108554   
TXN   0.660693  0.539648  0.219068  0.65607

In [14]:
def getAlphaBetaBenchmark(indices, tickers, cov, stats):
    #TODO consider adding risk free rate
    benchmark = {}
    for index in indices:
        alpha = []
        beta = []
        for ticker in tickers:
            indexVarience = cov[index][index]
            indexTickerCovarience = cov[index][ticker]
            indexTickerBeta = indexTickerCovarience / indexVarience
            indexTickerAlpha = stats[ticker]['TotalReturn'] - indexTickerBeta * stats[index]['TotalReturn']
            beta.append(indexTickerBeta)
            alpha.append(indexTickerAlpha)

        benchmark[index] = pd.DataFrame.from_dict({'Alpha' : alpha, 'Beta' : beta}, orient='index', columns=tickers)
    return benchmark

abBench = getAlphaBetaBenchmark(
    list(filter(None, indices.split(','))),
    list(filter(None, tickers.split(','))),
    cov, stats)

print(f"Alpa/Beta for SPY benchmark :\n{abBench['SPY']}")

Alpa/Beta for SPY benchmark :
           AAPL      TSLA      MSFT        GM       BCS      QCOM       PTR  \
Alpha  0.015872  1.721312 -0.172086  0.772418  0.437016  0.042966 -0.051308   
Beta   1.385885  1.981545  1.193959  1.146854  1.344935  1.446211  0.736107   

           VALE        GS        CI  ...        CB       LFC      MUFG  \
Alpha  1.165004  0.576771 -0.015219  ...  0.230395 -0.270370  0.156281   
Beta   1.160191  1.125696  0.909719  ...  0.905203  0.901178  0.783987   

           NVDA      ASML      ADBE      SONY       AMD       TXN         F  
Alpha -0.021738  0.521300 -0.200919  0.214130 -0.187396  0.064978  0.794848  
Beta   1.494100  1.352246  1.210740  0.641412  1.315700  1.286491  1.173361  

[2 rows x 21 columns]
